Hopefully, This script will allow me to load up and fix up the moored data

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import netCDF4 as nc
import datetime as dt
from salishsea_tools import evaltools as et, viz_tools
import gsw 
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import matplotlib.dates as mdates
import cmocean as cmo
import scipy.interpolate as sinterp
import pickle
import cmocean
import json
import f90nml
from collections import OrderedDict
from matplotlib.colors import LogNorm

fs=16
mpl.rc('xtick', labelsize=fs)
mpl.rc('ytick', labelsize=fs)
mpl.rc('legend', fontsize=fs)
mpl.rc('axes', titlesize=fs)
mpl.rc('axes', labelsize=fs)
mpl.rc('figure', titlesize=fs)
mpl.rc('font', size=fs)
mpl.rc('font', family='sans-serif', weight='normal', style='normal')

import warnings
#warnings.filterwarnings('ignore')
from IPython.display import Markdown, display

%matplotlib inline

In [2]:
saveloc='/ocean/kflanaga/MEOPAR/mooredData'
year=2015
Mooring='Dockton'

In [3]:
df = pd.read_csv(f'{saveloc}/{Mooring}_1_1_{year}_to_12_31_{year}.csv')

In [4]:
Lat=float(df['**Legend**'][20][24:32])
Lon=float(df['**Legend**'][20][39:49])

In [5]:
print(Lat)
print(Lon)

47.37611
-122.45722


In [6]:
df.drop(df.index[[tuple(range(0,48))]],inplace=True)

/home/kflanaga/miniconda3/envs/py39/lib/python3.9/site-packages/pandas/core/indexes/base.py:4114: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)


In [7]:
df.columns=df.iloc[0]

In [8]:
df.drop(df.index[[0]],inplace=True)
df.drop(df.index[[-1]],inplace=True)

In [9]:
df=df.reset_index()

In [10]:
df=df.drop('index',axis=1)
df=df.drop('Sonde_ID',axis=1)
df=df.drop('Sonde_Batt_V',axis=1)
df=df.drop('Logger_Batt_V',axis=1)

In [11]:
df.columns

Index(['Date', 'Depth_m', 'Qual_Depth', 'Water_Temperature_degC',
       'Qual_Water_Temperature', 'Salinity_PSU', 'Qual_Salinity',
       'Chlorophyll_Fluorescence_ug/L', 'Qual_Chlorophyll_Fluorescence'],
      dtype='object', name=48)

In [12]:
df[df.columns[1:-1]]=df[df.columns[1:-1]].apply(lambda col:pd.to_numeric(col, errors='coerce'))

In [13]:
df['Depth_m']

0        7.26
1        7.43
2        7.54
3        7.66
4        7.79
         ... 
34918    7.44
34919    7.36
34920    7.28
34921    7.21
34922    7.10
Name: Depth_m, Length: 34923, dtype: float64

In [14]:
df['Z']=df['Depth_m']
#df.head()

In [15]:
df['dtUTC']=pd.to_datetime(df['Date'],format='%m/%d/%Y %I:%M:%S %p')
df['Lat']=Lat
df['Lon']=Lon
df['Z']=df['Depth_m']
press=gsw.p_from_z(-1*df['Z'],df['Lat'])
df['SA']=gsw.SA_from_SP(df['Salinity_PSU'],press,
                       df['Lon'],df['Lat'])
df['CT']=gsw.CT_from_t(df['SA'],df['Water_Temperature_degC'],press)
df['Chl']=df['Chlorophyll_Fluorescence_ug/L']
df['YD']=et.datetimeToYD(df['dtUTC'])

In [16]:
dfg=df.groupby(by='YD')

In [17]:
df_daily_avg=dfg.mean()
df_daily_avg['Lat']=Lat
df_daily_avg['Lon']=Lon

In [19]:
df_daily_avg

48,Depth_m,Qual_Depth,Water_Temperature_degC,Qual_Water_Temperature,Salinity_PSU,Qual_Salinity,Chlorophyll_Fluorescence_ug/L,Z,Lat,Lon,SA,CT,Chl
YD,,,,,,,,,,,,,
1,7.133854,111.0,9.305000,111.0,28.590625,313.0,3.902083,7.133854,47.37611,-122.45722,28.728168,9.403922,3.902083
2,7.166354,111.0,8.936250,111.0,28.265521,313.0,5.617708,7.166354,47.37611,-122.45722,28.401499,9.037190,5.617708
3,7.191458,111.0,8.766458,111.0,28.220729,313.0,4.513542,7.191458,47.37611,-122.45722,28.356491,8.866717,4.513542
4,7.234792,111.0,8.913542,111.0,28.318437,313.0,5.042708,7.234792,47.37611,-122.45722,28.454668,9.013627,5.042708
5,7.315729,111.0,8.914375,111.0,28.054792,313.0,3.234375,7.315729,47.37611,-122.45722,28.189751,9.018700,3.234375
...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,7.507292,111.0,10.559479,111.0,28.811146,323.0,1.177083,7.507292,47.37611,-122.45722,28.949738,10.664677,1.177083
362,7.449375,111.0,10.550937,111.0,28.842292,323.0,1.237500,7.449375,47.37611,-122.45722,28.981035,10.655506,1.237500
363,7.413750,111.0,10.556875,111.0,28.859896,323.0,1.336458,7.413750,47.37611,-122.45722,28.998725,10.661171,1.336458


In [18]:
##### Saving data as Pickle files to be used in the summary file
saveloc='/ocean/kflanaga/MEOPAR/mooredData'
with open(os.path.join(saveloc,f'data_{Mooring}_{year}.pkl'),'wb') as hh:
    pickle.dump(df,hh)
    
with open(os.path.join(saveloc,f'daily_data_{Mooring}_{year}.pkl'),'wb') as hh:
    pickle.dump(df_daily_avg,hh)

Use group by yearday and then apply mean to everything. Oooohhh. That causes a problem. 